# AI Project: Question Answering with Mistral-7B-Instruct-v0.2


## This project implements a question-answering system using the Mistral-7B-Instruct-v0.2 model, which provides accurate and contextually relevant responses to user queries. The model has been integrated into a web application with Gradio serving as the frontend and FastAPI managing the backend processes. Users can input their questions through the Gradio interface, which communicates with the FastAPI server to retrieve answers generated by the Mistral-7B model. This setup demonstrates how advanced language models can be utilized in practical applications for real-time interaction and information retrieval.

In [1]:
#Installing libraries
!pip install fastapi uvicorn langchain-huggingface huggingface_hub langchain
!pip install nest-asyncio pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 716.1 kB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_6

In [2]:
#Importing libraries
from google.colab import userdata
from fastapi import FastAPI
from langchain_huggingface import HuggingFaceEndpoint
import os
from pyngrok import ngrok
import nest_asyncio
import uvicorn
from threading import Thread


In [3]:
#ngrok authentication key
!ngrok config add-authtoken 2kK5rhhIicPSj3LKoEND0O5N7cd_4EuzVYD9FH4TGQfZQA3aM

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [4]:
# Initialize necessary configurations
nest_asyncio.apply()

In [5]:
# Retrieve and set the Hugging Face API token from Colab's userdata
sec_key = userdata.get("HUGGINGFACEHUB_API_TOKEN")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = sec_key

In [6]:
# Define the HuggingFaceEndpoint
repo_id = "mistralai/Mistral-7B-Instruct-v0.2"
llm = HuggingFaceEndpoint(repo_id=repo_id, max_length=128, temperature=0.7, token=sec_key)

                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.
                    token was transferred to model_kwargs.
                    Please make sure that token is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [7]:
# Initialize FastAPI app
app = FastAPI()
@app.get("/query/")
def query_model(query: str):
    response = llm.invoke(query)
    return {"response": response}

In [8]:
# Set up ngrok tunnel
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

# Function to run the FastAPI app
def run_app():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Run the FastAPI app in a separate thread
thread = Thread(target=run_app)
thread.start()

Public URL: NgrokTunnel: "https://a0a2-34-125-191-64.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [898]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


# Gradio for frontend

In [9]:
#Installing libraries
!pip install gradio requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 99.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing installation: tomlkit 0.13.0
    Uninstalling tomlkit-0.13.0:
      Successfully uninstalled tomlkit-0.13.0


In [10]:
#Importing libraries
import gradio as gr
import requests

In [11]:
# Setting ngrok public URL printed from the FastAPI backend
ngrok_url = "https://a0a2-34-125-191-64.ngrok-free.app"

In [12]:
# Define the function to call FastAPI endpoint
def ask_question(question):
    response = requests.get(f"{ngrok_url}/query/", params={"query": question})
    return response.json().get("response")

In [13]:
# Create the Gradio interface
iface = gr.Interface(
    fn=ask_question,
    inputs="text",
    outputs="text",
    title="Question Answering with Mistral-7B",
    description="Ask any question and get an answer from the Mistral-7B model."
)

# Launch the interface
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://99c8b38231ad0a9414.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
